In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [2]:
train_df = pd.read_csv('./sub_train.csv')
test_df = pd.read_csv('./sub_test.csv')

In [3]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [ ]:
X = train_df[main_attrs].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train(x, y):
    print(x.shape, y.shape)
    
    models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = RandomForestRegressor(n_estimators=300,max_depth=None)
        clf.fit(train_x, train_y)
        print("Val score:", mean_squared_error(clf.predict(val_x), val_y, squared=False))
        models.append(clf)
    
    return models
models = train(X, y)

preds = []
for model in tqdm(models):
    preds.append(model.predict(test_df[main_attrs].values))
s = 0
for pred in preds:
    s += pred
s = s/5
result_df = pd.read_csv('./sample_solution.csv')
result_df['contest-tmp2m-14d__tmp2m'] = s
result_df.to_csv('submission_rf_v1.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:06,  1.32s/it]


(375734, 59) (375734,)


 20%|████████████████▍                                                                 | 1/5 [1:16:09<5:04:36, 4569.19s/it]

Val score: 0.4459941206619798


 40%|████████████████████████████████▊                                                 | 2/5 [2:32:32<3:48:53, 4577.74s/it]

Val score: 0.4433083463027348


 60%|█████████████████████████████████████████████████▏                                | 3/5 [3:48:26<2:32:13, 4566.64s/it]

Val score: 0.44326408165351733


 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [5:04:26<1:16:04, 4564.12s/it]

Val score: 0.4448287918571188
